# Trains and  tests on parts of  NA12878 data for Del_st, Del_end and No_sv, ground  truth Mills_nanosv

## Load dependences and setting output configuration

In [3]:
import os
import numpy as np
from keras.utils.np_utils import to_categorical
import keras
import gzip
from collections import Counter

%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


## Load data from npy files

In [4]:
# import pickle
# dico = pickle.load(open("/hpc/cog_bioinf/ridder/users/lsantuari/Processed/Test/060818/TestData_060818/NA12878/MultiLabelData/labels.pickle", "rb"))
# labels = dataset = dico["Mills2011_nanosv"]

#sample_name = 'NA12878'

sample_name = 'NA12878'

#date = '201118'
date = '231118'

#dico = pickle.load(open("/hpc/cog_bioinf/ridder/users/lsantuari/Processed/Test/060818/TestData_060818/PATIENT1/MultiLabelData/labels.pickle", "rb"))
dico_file = '/hpc/cog_bioinf/ridder/users/lsantuari/Processed/Test/'+date+'/TestData_'+date+'/'+sample_name+'/MultiLabelData/labels.pickle.gz'

with gzip.GzipFile(dico_file, "rb") as f:
    dico = np.load(f)
f.close()

print(dico.keys())

window_size = 200

dict_keys(['nanosv', 'nanosv_manta', 'manta', 'delly', 'lumpy', 'gridss', 'Mills2011', 'Mills2011_nanosv', 'nanosv_Mills2011', 'Mills2011_nanosv_manta', 'Mills2011_PacBio_Moleculo', 'Mills2011_PacBio_Moleculo_nanosv', 'nanosv_Mills2011_PacBio_Moleculo', 'Mills2011_PacBio_Moleculo_nanosv_manta', 'Mills2011_PacBio_Moleculo_Lumpy_GASVPro_DELLY_Pindel', 'id'])


In [5]:
# Load channel data and labels by chromosome

#Leaving out chromosome Y and MT for the moment
chr_list = list(map(str, np.arange(4,23)))
chr_list.append('X')

#chr_list = ['22']
print(chr_list)

data_original = []
labels_original = []
#label_type = 'Mills2011_nanosv'
label_type = 'Mills2011_nanosv'
datapath = '/hpc/cog_bioinf/ridder/users/lsantuari/Processed/Test/'+date+'/TestData_'+date+'/'+sample_name+'/ChannelData/'

for i in chr_list:
#for i in [16, 17, 18]:
    
    print('Loading data for Chr%s' % i)
    data_file = datapath + sample_name + '_' + str(i) + '.npy.gz'
    with gzip.GzipFile(data_file, "rb") as f:
        data_mat = np.load(f)
        data_original.extend(data_mat)
    f.close()
    
    labels_original.extend(dico[label_type][i])
    
print(Counter(labels_original))
assert len(data_original) == len(labels_original)

['4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X']
Loading data for Chr4
Loading data for Chr5
Loading data for Chr6
Loading data for Chr7
Loading data for Chr8
Loading data for Chr9
Loading data for Chr10
Loading data for Chr11
Loading data for Chr12
Loading data for Chr13
Loading data for Chr14
Loading data for Chr15
Loading data for Chr16
Loading data for Chr17
Loading data for Chr18
Loading data for Chr19
Loading data for Chr20
Loading data for Chr21
Loading data for Chr22
Loading data for ChrX
Counter({'noSV': 925494, 'UK': 1475, 'DEL_start': 1051, 'DEL_end': 949})


In [6]:
for i in chr_list:
    print(Counter(dico[label_type][i]))

Counter({'noSV': 81908, 'UK': 161, 'DEL_end': 116, 'DEL_start': 112})
Counter({'noSV': 56835, 'DEL_start': 114, 'DEL_end': 100, 'UK': 100})
Counter({'noSV': 61932, 'UK': 151, 'DEL_start': 87, 'DEL_end': 75})
Counter({'noSV': 67146, 'UK': 92, 'DEL_start': 91, 'DEL_end': 72})
Counter({'noSV': 49394, 'UK': 98, 'DEL_start': 76, 'DEL_end': 59})
Counter({'noSV': 47574, 'DEL_end': 66, 'DEL_start': 61, 'UK': 49})
Counter({'noSV': 72671, 'UK': 96, 'DEL_start': 57, 'DEL_end': 50})
Counter({'noSV': 50178, 'UK': 86, 'DEL_end': 60, 'DEL_start': 59})
Counter({'noSV': 51421, 'UK': 72, 'DEL_end': 58, 'DEL_start': 56})
Counter({'noSV': 31937, 'DEL_start': 58, 'DEL_end': 48, 'UK': 42})
Counter({'noSV': 31267, 'UK': 64, 'DEL_start': 38, 'DEL_end': 28})
Counter({'noSV': 29985, 'DEL_start': 32, 'UK': 30, 'DEL_end': 29})
Counter({'noSV': 56677, 'UK': 125, 'DEL_start': 27, 'DEL_end': 20})
Counter({'noSV': 41098, 'UK': 68, 'DEL_end': 25, 'DEL_start': 23})
Counter({'noSV': 33309, 'UK': 51, 'DEL_start': 40, 'DE

In [7]:
labels_original = []
label_type = 'Mills2011_nanosv'
#label_type = 'nanosv_manta'

for i in chr_list:
    #print('Loading data for Chr%s' % i)
    labels_original.extend(dico[label_type][i])
    
print(Counter(labels_original))

id_original = []
for i in chr_list:
    #print('Loading data for Chr%s' % i)
    id_original.extend(dico['id'][i])
id_original = [str(v['chromosome'])+':'+str(v['position']) for v in id_original]
# print(id_original)

labels_original = np.array(labels_original)

Counter({'noSV': 925494, 'UK': 1475, 'DEL_start': 1051, 'DEL_end': 949})


In [8]:
labels_svcaller = {}
#label_type = 'nanosv_Mills2011'

for label_type in ['manta', 'gridss', 'lumpy', 'delly']:
    labels_svcaller[label_type] = []
    for chr in chr_list:
        labels_svcaller[label_type].extend(dico[label_type][chr])
    labels_svcaller[label_type] = np.array(labels_svcaller[label_type])

manta_gridss = (labels_svcaller['manta']==labels_svcaller['gridss'])
lumpy_delly = (labels_svcaller['lumpy']==labels_svcaller['delly'])
#manta_nanosv = (labels_svcaller['manta']==labels_svcaller['nanosv'])
manta_gridss_lumpy_delly = (manta_gridss == lumpy_delly)
#manta_gridss_lumpy_delly_nanosv = (manta_gridss_lumpy_delly == manta_nanosv)

print(Counter(manta_gridss_lumpy_delly))

labels = []
for i in np.arange(len(labels_svcaller['manta'])):
    if manta_gridss_lumpy_delly[i]:
            labels.append(labels_svcaller['manta'][i])
    else:
        labels.append('UK')

print(Counter(labels_svcaller['manta']))
print(Counter(labels))

assert len(labels_svcaller['manta']) == len(labels)
labels = np.array(labels)

Counter({True: 898125, False: 30844})
Counter({'noSV': 916473, 'UK': 5380, 'DEL_start': 3564, 'DEL_end': 3552})
Counter({'noSV': 894285, 'UK': 32516, 'DEL_start': 1111, 'DEL_end': 1057})


In [9]:
data_original = np.array(data_original)

print(data_original.shape)
print(len(labels_original))

(928969, 44, 200)
928969


In [10]:
n_windows = 30
number_channels = data_original.shape[1]
label = ["None"] * number_channels

In [11]:
# Fill labels for legend

label[0] = "coverage"
label[1] = "#left clipped reads"
label[2] = "#right clipped reads"
label[3] = "INV_before"
label[4] = "INV_after"
label[5] = "DUP_before"
label[6] = "DUP_after"
label[7] = "TRA_opposite"
label[8] = "TRA_same"

i = 9
for direction in ['Forward','Reverse']:
    for clipped in ['Left','Right','Not']:
        for value in ['sum', 'num', 'median', 'outliers']:
            label[i] = direction+'_'+clipped+'_Clipped_'+value
            i = i+1

label[i] = "#left split reads"
i = i+1
label[i] = "#right split reads"
i = i+1

for clipped in ['L','R']:
    for value in ['sum', 'num', 'median']:
        label[i] = clipped+'_SplitRead_'+value
        i = i+1
        
label[i] = "GC"
i = i+1
label[i] = "Mappability"
i = i+1
label[i] = "One_hot_Ncoding"

for k,l in enumerate(label):
    print(str(k)+':'+l)

0:coverage
1:#left clipped reads
2:#right clipped reads
3:INV_before
4:INV_after
5:DUP_before
6:DUP_after
7:TRA_opposite
8:TRA_same
9:Forward_Left_Clipped_sum
10:Forward_Left_Clipped_num
11:Forward_Left_Clipped_median
12:Forward_Left_Clipped_outliers
13:Forward_Right_Clipped_sum
14:Forward_Right_Clipped_num
15:Forward_Right_Clipped_median
16:Forward_Right_Clipped_outliers
17:Forward_Not_Clipped_sum
18:Forward_Not_Clipped_num
19:Forward_Not_Clipped_median
20:Forward_Not_Clipped_outliers
21:Reverse_Left_Clipped_sum
22:Reverse_Left_Clipped_num
23:Reverse_Left_Clipped_median
24:Reverse_Left_Clipped_outliers
25:Reverse_Right_Clipped_sum
26:Reverse_Right_Clipped_num
27:Reverse_Right_Clipped_median
28:Reverse_Right_Clipped_outliers
29:Reverse_Not_Clipped_sum
30:Reverse_Not_Clipped_num
31:Reverse_Not_Clipped_median
32:Reverse_Not_Clipped_outliers
33:#left split reads
34:#right split reads
35:L_SplitRead_sum
36:L_SplitRead_num
37:L_SplitRead_median
38:R_SplitRead_sum
39:R_SplitRead_num
40:R_Spl

## Plot the windows per class

In [12]:
datapath_fig = datapath+'../Figures'
os.mkdir(datapath_fig)

FileExistsError: [Errno 17] File exists: '/hpc/cog_bioinf/ridder/users/lsantuari/Processed/Test/231118/TestData_231118/NA12878/ChannelData/../Figures'

In [ ]:
import math
import matplotlib.pyplot as plt


def plot_channels(indices, n_windows, output_path):
    #for i in range (start_window, start_window + n_windows):
    for idx in np.arange(n_windows):
        
        i = indices[idx]
        #print(y_train[i], 'id:', i)
        fig_name = labels_original[i]+'_'+'Chr'+id_original[i].replace(':','-')
        
        print(labels_original[i], 'id:', 'Chr'+id_original[i])
        plt.title('Class: '+labels_original[i]+' '+'Position: Chr'+id_original[i], fontsize=30)
        plt.ylim([0, number_channels])
        plt.yticks(np.arange(number_channels), label, fontsize=15)
        plt.xticks(fontsize=30)
        
        plt.vlines(x=0, ymin=0, ymax=number_channels, color='black')
        
        #fig, axs = plt.subplots(number_channels, 1)
        
        #for j in range (number_channels-1, -1, -1):
        #for j in range (number_channels-1, -1, -1):
        for j in range (number_channels-1, -1, -1):
        #for j in [12,16,20,24,28,32]:
            mul = 1
            shift = 0
            start = 0
            #if j in [0,1,4,5]:
                #shift = -60
            #if j in [4,5,6,7]:
                #start = 70
            #print(X_train[i][j])
            #X_win = data_original[i][j]
            if sum(data_original[i][j])!=0:
                X_win = ((data_original[i][j]-min(data_original[i][j]))/max(data_original[i][j]))
                #X_win = data_original[i][j]
            else:
                X_win = data_original[i][j]
            #Z = [start + shift + x + 5*j*4 for x in X_train[i][j]] 
            Z = [x + j for x in X_win] 
            
            plt.plot(np.arange(-100,100), Z, label = label[j], linewidth=2)
            plt.fill_between(np.arange(-100,100), j, Z, alpha=.5)
            
            #axs[j].plot(Z, label = label[j], linewidth=2)
            #plt.setp(axs[j].get_xticklabels(), visible=False)
            
            #plt.fill_between(np.arange(len(Z)), 1, Z, alpha=1)
            
            #plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., prop={'size': 10})
        #fig.tight_layout()
        #plt.show()
        plt.savefig(os.path.join(output_path, fig_name+'.png'))
        plt.clf()
        #
        

In [ ]:
svtype = 'DEL'
idx = list(np.where(labels==svtype+"_end")[0])
i = idx[0]
j = 0
X_win = ((data_original[i][j]-min(data_original[i][j]))/max(data_original[i][j]))
#print(data_original[i][j])
#print(X_win+1)
print('min:%d, max:%d' % (X_win.min(), X_win.max()))

In [ ]:
#idx = np.where(labels=="noSV")
#idx = np.where(labels=="noSV")
svtype = 'DEL'
#idx = np.where(labels_original==svtype+"_start")[0]
class_label = svtype+"_start"

for class_label in ['DEL_start', 'DEL_end', 'noSV']:
    
    idx = list(np.where(labels_original==class_label)[0])
    #print(idx)
    #idx = np.where(labels==svtype+"_end")

    #print(idx)

    datapath_fig_class = os.path.join(datapath_fig, class_label)
    if not os.path.exists(datapath_fig_class):
        os.mkdir(datapath_fig_class)

    # start_window = 3990
    plt.rcParams['figure.figsize'] = (30,20)
    plot_channels(idx, 30, datapath_fig_class)
    #print(labels[0])

In [ ]:
# for ii, ch_name in enumerate(label):
#     print('%d => %s'%(ii, ch_name))

In [ ]:
import os
from datetime import datetime
timestr = datetime.now().strftime("%Y%m%d-%H%M%S")
datapath_now = datapath+'../ModelData/'+timestr
os.mkdir(datapath_now)

## Remove windows flagged as 'UK' (unknown)

In [ ]:
#del data
keep = np.where(np.array(labels_original)!='UK')
#keep = np.where(np.array(labels)!='UK')

data = data_original[keep]
labels = labels_original[keep]
#labels = labels[keep]

In [ ]:
idx = np.arange(0, 9)
# # outliers channels
#idx = np.append(idx, [12,16,20,24,28,32], 0)
# # split reads
idx = np.append(idx, [33,34], 0)
#idx = np.append(idx, [42,43], 0)
print(idx)

data = data[:,idx,:]

In [ ]:
file = open(datapath_now+'/Channels.txt','w') 

file.write('Using the following channels:\n\n')
for k,lab_i in enumerate(idx):
    file.write(str(k)+':'+label[lab_i]+'\n')

file.close() 

In [ ]:
cnt_lab = Counter(labels)
min_v = min([v for k, v in cnt_lab.items()])
max_v = max([v for k, v in cnt_lab.items()])

print(cnt_lab)
print('Minimum number of labels = ' + str(min_v))
print('Maximum number of labels = ' + str(max_v))

In [ ]:
data_balanced = []
labels_balanced = []

#class_size = 10**5

for l in cnt_lab.keys():
    #print(l)
    iw = np.where(labels==l)
    #print(iw[0])
    #ii = np.random.choice(a=iw[0], size=min_v, replace=False)
    ii = iw[0][:min_v]
    #ii = np.random.choice(a=iw[0], size=class_size, replace=True)
    #print(ii)
    data_balanced.extend(data[ii])
    labels_balanced.extend(labels[ii])

print(Counter(labels_balanced))

X = np.array(data_balanced)
y = np.array(labels_balanced)

In [ ]:
idx = numpy.unique(np.where(np.isnan(X))[0])
X = np.delete(X, idx, 0)
y = np.delete(y, idx, 0)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
number_channels = X.shape[1]
number_channels

In [ ]:
#Split into training, validation and test set 60/20/20

cnt_lab = Counter(y)
n_lab = [v for v in cnt_lab.values()][0]
i_train = int(n_lab*0.6)
i_val = i_train + int(n_lab*0.2)
print(i_train)
print(i_val)

X_train = []
y_train = []
X_val = []
y_val = []
X_test = []
y_test = []

for l in cnt_lab.keys():
    iw = np.where(y==l)
    #print(iw[0])
    #print(iw[0][:i_train])
    X_train.extend(X[iw[0][:i_train]])
    X_val.extend(X[iw[0][i_train:i_val]])
    X_test.extend(X[iw[0][i_val:]])

    y_train.extend(y[iw[0][:i_train]])
    y_val.extend(y[iw[0][i_train:i_val]])
    y_test.extend(y[iw[0][i_val:]])

X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

In [ ]:
assert X.shape[0] == y.shape[0]
print(int(X.shape[0]/10))

In [ ]:
from collections import defaultdict

def split_dataset(X, y, cv_fold):
    
    cnt_lab = Counter(y)
    print(cnt_lab)
    n_lab = [v for v in cnt_lab.values()][0]

    fold_count = int(n_lab/cv_fold)
    
    res_X = defaultdict(dict)
    res_y = defaultdict(dict)
    
    X_train_val = []
    
    for l in cnt_lab.keys():
        iw = np.where(y==l)
        for i in np.arange(cv_fold):
            X_train_val.extend(X[iw[0][i*fold_count:(i+1)*fold_count]])
            
            
    
split_dataset(X, y, 10)

   
## Transposes every window in X, to comply to McFly format

In [ ]:
def transposeDataset(X):
    image = []
    for i in range (0, len(X -1)):
        tr = X[i].transpose()
        image.append(tr)
    return np.array(image)

image1 = transposeDataset(X_train)
image2 = transposeDataset(X_val)
image3 = transposeDataset(X_test)
X_train = image1
X_val = image2
X_test = image3

n_channels = len(X_train[0][0])
print(len(X_test[0][0]))

## Prepare the labels in McFly format

In [ ]:
mapclasses = {'DEL_start': 1, 'DEL_end': 0,  'noSV': 2}
print(mapclasses)

In [ ]:
y_train = np.array([mapclasses[c] for c in y_train], dtype='int')
y_val = np.array([mapclasses[c] for c in y_val], dtype='int')
y_test = np.array([mapclasses[c] for c in y_test], dtype='int')
y_train_binary = to_categorical(y_train)
y_val_binary = to_categorical(y_val)
y_test_binary = to_categorical(y_test)

In [ ]:
y_test_binary

In [ ]:
y_val_binary

In [ ]:
X_train.shape

##  Generate and train neural networks 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution1D, Flatten, MaxPooling1D
from keras.optimizers import Adam
import sys
import os
import numpy as np
sys.path.insert(0, os.path.abspath('..'))
from mcfly import modelgen, find_architecture

In [ ]:
import inspect
lines = inspect.getsource(modelgen)
# print(lines)
sfile = inspect.getsourcefile(modelgen)
# print(sfile)

In [ ]:
np.random.seed = 321
num_classes = y_train_binary.shape[1]

In [ ]:
%%time
models = modelgen.generate_models(X_train.shape,
                                  num_classes,
                                  number_of_models = 10,
                                  model_type = 'CNN',
                                  cnn_min_layers=2,
                                  cnn_max_layers=6,
                                  cnn_min_filters = 4, 
                                  cnn_max_filters = 8, 
                                  cnn_min_fc_nodes=6, 
                                  cnn_max_fc_nodes=12,
                                  low_lr=2, high_lr=4,
                                  kernel_size = 7)

# models = modelgen.generate_models(X_train.shape,
#                                   num_classes,
#                                   number_of_models = 1,
#                                   model_type = 'CNN',
#                                   cnn_min_layers=2,
#                                   cnn_max_layers=2,
#                                   cnn_min_filters = 4, 
#                                   cnn_max_filters = 4, 
#                                   cnn_min_fc_nodes=6, 
#                                   cnn_max_fc_nodes=6,
#                                   low_lr=2, high_lr=2,
#                                   kernel_size = 7)

In [ ]:
%%time
i=0
for model, params, model_types in models:
    print('model ' + str(i))
    i=i+1
    print(params)
    model.summary()

In [ ]:
train_set_size = X_train.shape[0]
train_set_size

In [ ]:
# for i in np.arange(X_train.shape[2]):
#     print(X_train[4597,:,i])

In [ ]:
#assert not np.any(np.isnan(X_train))
#print(numpy.unique(np.where(np.isnan(X_train))[0]))
#print(X_train[101])

In [ ]:
%%time
histories, val_accuracies, val_losses = find_architecture.train_models_on_samples(X_train, y_train_binary,
                                                                                 X_val, y_val_binary,
                                                                                 models, nr_epochs=1, 
                                                                                 subset_size=train_set_size,
                                                                                 verbose=True)

## Save the best model

In [ ]:
best_model_index = np.argmax(val_accuracies)
best_model, best_params, best_model_types = models[best_model_index]
print(best_model_index, best_model_types, best_params)

In [ ]:
from keras.models import load_model

model_path = os.path.join(datapath_now, 'model')

best_model.save(model_path)

## Train the best model on the full dataset

In [ ]:
#We make a copy of the model, to start training from fresh
nr_epochs = 1
datasize = train_set_size # Change in `X_train.shape[0]` if training complete data set
history = best_model.fit(X_train, y_train_binary,
              epochs=nr_epochs, validation_data=(X_val, y_val_binary))

## Reload the trained model

In [ ]:
# import os
# from keras.models import load_model
# model_path = os.path.join(datapath+'../ModelData',  'Gtcg_13_9')
# model_reloaded = load_model(model_path)

## Inspect model predictions on test data

In [ ]:
best_model.save(model_path)

model = best_model 
datasize = X_test.shape[0]
probs = model.predict_proba(X_test[:datasize,:,:],batch_size=1000, verbose=True)

## Columns are predicted labels by DeepSV, rows are nanopore labels

In [ ]:
mapclasses = {'DEL_end': 0, 'DEL_start': 1, 'noSV': 2}
dict=mapclasses
dict_sorted = sorted(dict.items(), key=lambda x: x[1])
dict_sorted

In [ ]:
labels = [i[0] for i in dict_sorted]

In [ ]:
import pandas as pd
predicted = probs.argmax(axis=1)
y_index = y_test_binary.argmax(axis=1)
confusion_matrix = pd.crosstab(pd.Series(y_index), pd.Series(predicted))
confusion_matrix.index = [labels[i] for i in confusion_matrix.index]
confusion_matrix.columns = [labels[i] for i in confusion_matrix.columns]
confusion_matrix.reindex(columns=[l for l in labels], fill_value=0)
print(confusion_matrix)
confusion_matrix.to_csv(path_or_buf=os.path.join(datapath_now,'model_confusion_matrix.csv'))

## Test best current model on testset

### Overall accuracy on the test set:

In [ ]:
%%time
score_test = model.evaluate(X_test, y_test_binary, verbose=False)
print('Test loss and accuracy of best model: ' + str(score_test))

In [ ]:
df_conf = pd.DataFrame()

for i in np.linspace(1.0 / len(labels), 1, num=50, endpoint=False):

    predicted = np.argwhere(probs > i)[:, 1]
    y_index = np.argwhere(y_test_binary > i)[:, 1]

    # Rows: true, columns: predicted
    confusion_matrix = pd.crosstab(pd.Series(y_index), pd.Series(predicted))
    confusion_matrix.index = [labels[i] for i in confusion_matrix.index]
    confusion_matrix.columns = [labels[i] for i in confusion_matrix.columns]
    confusion_matrix.reindex(columns=[l for l in labels], fill_value=0)

    #print('Confusion matrix:')
    #print(confusion_matrix)
    
    for l in labels:
        if l in confusion_matrix.index:
            # print(confusion_matrix.loc[l,:])
            # print(confusion_matrix.loc[:,l])

            # label_correct = confusion_matrix.loc[l, l]
            label_precision = np.around(confusion_matrix.loc[l, l] / sum(confusion_matrix.loc[:, l]) * 100)
            label_recall = np.around(confusion_matrix.loc[l, l] / sum(confusion_matrix.loc[l, :]) * 100)
            label_F1 = 2 * (label_precision * label_recall) / (label_precision + label_recall)

            # print(f'Iter:{i} {l} -> Precision:{label_precision}%, Recall:{label_recall}%, F1:{label_F1}')

            df_intres = pd.DataFrame(
                {'iteration': [i], 'label': [l],
                 'precision': [label_precision], 'recall': [label_recall], 'F1': [label_F1]})
            df_conf = df_conf.append(df_intres)
            
# print(df_conf)
df_conf.to_csv(path_or_buf=os.path.join(datapath_now,'model_PrecRec.csv'))

In [ ]:
# df_conf[df_conf['label']=='DEL_start']

df_conf = pd.read_csv(filepath_or_buffer=os.path.join(datapath_now,'model_PrecRec.csv'))

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
fig.set_figheight(5)
fig.set_figwidth(15)
idx = 1
for lab in labels:
    ax = plt.subplot(1,3,idx)
    for measure in ['precision', 'recall', 'F1']:
        ax.plot(df_conf[df_conf['label']==lab]['iteration'],
                 df_conf[df_conf['label']==lab][measure])
    ax.set_title('Label: '+lab)
    ax.set_ylabel('')
    ax.set_xlabel('probability cutoff')
    ax.legend()
    idx += 1

fig.savefig(os.path.join(datapath_now,'model_PrecRec.png'))

## Generating predictions for PATIENT1

In [ ]:
from keras.models import load_model
datapath_now = os.path.join(datapath, '../../NA12878/ModelData/20181126-104011')
model_path = os.path.join(datapath_now,  'model')
model = load_model(model_path)

In [ ]:
# Load channel data and labels by chromosome

sample_name = 'PATIENT1'
date = '231118'

#Leaving out chromosome Y and MT for the moment
chr_list = list(map(str, np.arange(1,23)))
chr_list.append('X')
#chr_list = ['22']
print(chr_list)

data_original = []
datapath = '/hpc/cog_bioinf/ridder/users/lsantuari/Processed/Test/'+date+'/TestData_'+date+'/'+sample_name+'/ChannelData/'

for i in chr_list:
#for i in [16, 17, 18]:
    
    print('Loading data for Chr%s' % i)
    data_file = datapath + sample_name + '_' + str(i) + '.npy.gz'
    with gzip.GzipFile(data_file, "rb") as f:
        data_mat = np.load(f)
        data_original.extend(data_mat)
    f.close()

data_original = np.array(data_original)

In [ ]:
data = data_original

idx = np.arange(0, 9)
# # outliers channels
idx = np.append(idx, [12,16,20,24,28,32], 0)
# # split reads
idx = np.append(idx, [33,34], 0)
#idx = np.append(idx, [42,43], 0)
print(idx)

data = data[:,idx,:]

In [ ]:
X = np.array(data)
y = np.array(labels)

In [ ]:
idx = numpy.unique(np.where(np.isnan(X))[0])
X = np.delete(X, idx, 0)
y = np.delete(y, idx, 0)
id_label = np.delete(np.array(id_original), idx, 0)


In [ ]:
X = transposeDataset(X)

In [ ]:
%%time
## Inspect model predictions on test data
datasize = X.shape[0]
probs = model.predict_proba(X, batch_size=1000, verbose=True)

In [ ]:
mapclasses = {'DEL_end': 0, 'DEL_start': 1, 'noSV': 2}
dict=mapclasses
dict_sorted = sorted(dict.items(), key=lambda x: x[1])
dict_sorted
labels = [i[0] for i in dict_sorted]

np.save(file=os.path.join(datapath_now,sample_name+'_probs.npy'), arr=probs)
predicted = probs.argmax(axis=1)
y_pred = [labels[i] for i in predicted]
print(Counter(y_pred))

Write results in BED file

In [ ]:
file = open(os.path.join(datapath_now,sample_name+'_predictions.bed'),'w') 

for k,lab_i in enumerate(y_pred):
    coord = id_label[k].split(':')
    file.write(str(coord[0])+'\t'+str(int(coord[1])-1)\
               +'\t'+str(coord[1])+'\t'+lab_i+'\n')

file.close() 